In [433]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
%matplotlib inline

In [434]:
# Load .env enviroment variables
load_dotenv()



True

In [435]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [436]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"

# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_tickers

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.9000,169.00,168.6000,391,168.98,15303,168.871369,AAPL
2023-10-30 08:15:00+00:00,168.8500,169.00,168.8500,105,168.99,5249,168.955731,AAPL
2023-10-30 08:30:00+00:00,168.8500,168.92,168.8500,72,168.85,3911,168.863544,AAPL
2023-10-30 08:45:00+00:00,168.9900,168.99,168.7100,81,168.86,4184,168.869565,AAPL
2023-10-30 09:00:00+00:00,169.2300,169.23,168.9700,111,168.97,5241,169.137236,AAPL
...,...,...,...,...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,169.90,169.8200,41,169.82,4060,169.863674,AAPL
2023-10-30 23:00:00+00:00,169.8200,169.91,169.8200,112,169.87,5285,169.867104,AAPL
2023-10-30 23:15:00+00:00,169.8500,169.87,169.8500,97,169.85,3450,169.861542,AAPL


In [437]:
# Defining variables
shares = 100




In [438]:
# Reorganize the DataFrame
df_tickers.drop(['high','low','trade_count','volume','vwap','symbol','open'], axis=1, inplace=True)

In [439]:
### Establishing control benchmark
df_tickers['return'] = df_tickers['close'].pct_change() + 1
df_tickers['return'].iat[0] = 1
fund = shares*df_tickers['close'][0]
df_tickers['benchmark'] = fund * df_tickers['return'].cumprod()
df_tickers


,close,return,benchmark
timestamp,,,
2023-10-30 08:00:00+00:00,168.9000,1.000000,16890.00
2023-10-30 08:15:00+00:00,168.8500,0.999704,16885.00
2023-10-30 08:30:00+00:00,168.8500,1.000000,16885.00
2023-10-30 08:45:00+00:00,168.9900,1.000829,16899.00
2023-10-30 09:00:00+00:00,169.2300,1.001420,16923.00
...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,0.999823,16986.99
2023-10-30 23:00:00+00:00,169.8200,0.999706,16982.00
2023-10-30 23:15:00+00:00,169.8500,1.000177,16985.00


In [440]:
### Creating new Dataframe for MACD calculations
df_macd = pd.DataFrame()
df_macd['MACD'] =  df_tickers['close'].ewm(halflife=12).mean() - df_tickers['close'].ewm(halflife=26).mean()
df_macd['Signal'] = df_macd['MACD'].ewm(halflife=9).mean()
df_macd['histogram'] = df_macd['MACD'] - df_macd['Signal']
df_macd

,MACD,Signal,histogram
timestamp,,,
2023-10-30 08:00:00+00:00,0.000000,0.000000,0.000000
2023-10-30 08:15:00+00:00,-0.000389,-0.000202,-0.000187
2023-10-30 08:30:00+00:00,-0.000511,-0.000313,-0.000198
2023-10-30 08:45:00+00:00,0.001111,0.000085,0.001025
2023-10-30 09:00:00+00:00,0.005153,0.001261,0.003893
...,...,...,...
2023-10-30 22:45:00+00:00,0.121949,0.125345,-0.003396
2023-10-30 23:00:00+00:00,0.116158,0.124658,-0.008500
2023-10-30 23:15:00+00:00,0.111433,0.123669,-0.012236


In [441]:
### Plotting Close price

Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='Closing Price AAPL',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting MACD LINE & signal line
MACD_plot = df_macd['MACD'].hvplot.line(    
    x='timestamp', 
    y='MACD',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Signal_plot = df_macd['Signal'].hvplot.line(    
    x='timestamp', 
    y='Signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting Histogram
Histogram_plot = df_macd['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)


In [442]:
display(Close_plot)
display(MACD_plot * Signal_plot)
Histogram_plot

:Curve   [timestamp]   (close)

:Overlay
   .Curve.MACD   :Curve   [timestamp]   (MACD)
   .Curve.Signal :Curve   [timestamp]   (Signal)

:Bars   [timestamp]   (histogram)

### I HAVE CALCULATED THE MACD LINE THE SIGNAL LINE
### I WILL BE USING A CROSSOVER STRATEGY TO MAKE MONEY ON IT.
Investopedia
Crossover Strategy
A crossover occurs when the signal and MACD line cross each other. The MACD generates a bullish signal when it moves above its own nine-day EMA and sends a sell signal (bearish) when it moves below its nine-day EMA.

In [443]:
# Creating signals
# Finding long or short trades
df_macd["X"]=np.where(df_macd["MACD"]>df_macd["Signal"],1,0) 
df_macd["X"]=df_macd["X"].diff() 
df_macd['close'] = df_tickers['close']
display(df_macd)

# Extracting the timestamps that indicate bullish signals (Signals where we buy) and that indicate bearish signals (signals where we sell)
Xreturns=df_macd[df_macd["X"].isin([1,-1])] 


# Dataframe with trade signals and dates
Xreturns['cashflow'] = -Xreturns['X'] * Xreturns['close'] * shares
cashflow = Xreturns["cashflow"].sum()
display(Xreturns)
display(cashflow)

,MACD,Signal,histogram,X,close
timestamp,,,,,
2023-10-30 08:00:00+00:00,0.000000,0.000000,0.000000,NaN,168.9000
2023-10-30 08:15:00+00:00,-0.000389,-0.000202,-0.000187,0.0,168.8500
2023-10-30 08:30:00+00:00,-0.000511,-0.000313,-0.000198,0.0,168.8500
2023-10-30 08:45:00+00:00,0.001111,0.000085,0.001025,1.0,168.9900
2023-10-30 09:00:00+00:00,0.005153,0.001261,0.003893,0.0,169.2300
...,...,...,...,...,...
2023-10-30 22:45:00+00:00,0.121949,0.125345,-0.003396,-1.0,169.8699
2023-10-30 23:00:00+00:00,0.116158,0.124658,-0.008500,0.0,169.8200
2023-10-30 23:15:00+00:00,0.111433,0.123669,-0.012236,0.0,169.8500


c:\Users\Micha\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,MACD,Signal,histogram,X,close,cashflow
timestamp,,,,,,
2023-10-30 08:45:00+00:00,0.001111,0.000085,0.001025,1.0,168.9900,-16899.00
2023-10-30 12:00:00+00:00,0.006359,0.006699,-0.000341,-1.0,168.9300,16893.00
2023-10-30 13:30:00+00:00,0.017787,-0.000027,0.017815,1.0,170.8939,-17089.39
2023-10-30 22:45:00+00:00,0.121949,0.125345,-0.003396,-1.0,169.8699,16986.99


-108.39999999999782

In [444]:
### Cashflow is being calculated as the sum of all outflows and inflows of cash. The outflows are when we are buying, that inflows are when we are selling.
### The benchmark is being calculated as a result of the initial value of 100 shares defined in the fund variable minus the last value in the df_tickers['benchmark'] column.
print(f"Profit/Loss MACD: {cashflow}\nProfit/Loss Benchmark: {fund - df_tickers['benchmark'][-1]}")

Profit/Loss MACD: -108.39999999999782
Profit/Loss Benchmark: -80.00000000000364
